[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BouleJaune/picselliaT/blob/master/Classification_TF2.ipynb)

In [ ]:
!pip install picsellia

# Setting up your Picsell client

First let's impor Tensorflow and the Picsell.ia sdk

In [1]:
import tensorflow as tf
from picsellia import Client
import os

Let's set the name to your soon to come classification model and put your tokens here:

In [4]:
api_token = "5890ad3e6a9701cd123ef91b69ad396de8c2d20a"
project_token = "05a91847-53a6-4b29-9014-d372d11836c7" 
model_name = "Classification_COVID_Masks"

Now we need to initialize our client so we can communicate with the platform. 

In [6]:
clt = Client(api_token=api_token)
clt.checkout_project(project_token=project_token)
clt.create_network(model_name)

Welcome Xenio3, Glad to have you back
{'network_id': '52a29433-286b-4050-8f2a-93ac37df5d14', 'training_id': 0, 'checkpoints': {}}
First time using Picsell.ia on this project, initializing directories ...
Creating directory for project mask-classification/Classification_COVID_Masks/0
Creating directory for PNG Images of project mask-classification/Classification_COVID_Masks/0
Creating directory for checkpoints project mask-classification/Classification_COVID_Masks/0
Creating directory for logs of project mask-classification/Classification_COVID_Masks/0/metrics
Creating directory for records of project mask-classification/Classification_COVID_Masks/0
Creating directory for config of project mask-classification/Classification_COVID_Masks/0
Creating directory for results of project mask-classification/Classification_COVID_Masks/0/results
New Network have been created


Once the instance is created we can download the images and annotations and generate a label map.
The label map is just here to map the string value of the label to a more convenient label id.

The ```train_test_split()``` method is smartly splitting our data in two sets.  

In [7]:
clt.dl_annotations()
clt.generate_labelmap()
clt.train_test_split()
clt.dl_pictures()

Annotations pulled ...
Generating labelmap ...
Label_map.pbtxt crée @ mask-classification/Classification_COVID_Masks/0/label_map.pbtxt
90 Images used for training, 23 Images used for validation
Repartition send ..
[==================================================]0 files were already on your machine
 113 PNG images have been downloaded to your machine


# Data pre-processing

## Converting data into serialized TFRecord files

We want to serialize those images and labels inside a ```TFRecord``` format file. By doing so the data will be way more efficiently read by tensorflow. 

In order to do this we need to generate a ```tf.Example``` for each image which stores the image and its label as a protobuf, then we serialize and write those ```tf.Example``` objects inside the ```TFRecord``` file.

First we create some shortcut functions to wrap the features messages. Those functions convert standard TensorFlow types to a ```tf.Example```-compatible ```tf.train.Feature``` object. In our case we just want to store the encoded image and the label id.

In [8]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

We can create our .record files from there. 

To do so, we define a new function which will iterate for each set through each image and generate a ```tf.Example``` message that we'll write inside our file. 

We use the ```clt.tf_vars_generator``` method from the sdk to retrieve the data before converting them into the ```tf.Example``` message.

In [16]:
def create_record_files(label_map, record_dir, tfExample_generator):
    datasets = ["train", "eval"]    
    for dataset in datasets:
        output_path = os.path.join(record_dir, dataset+".record")
        writer = tf.io.TFRecordWriter(output_path)
        for variables in tfExample_generator(label_map, ensemble=dataset, annotation_type="classification"):
            (width, height, filename, encoded_jpg, image_format, 
                classes_text, classes) = variables

            tf_example = tf.train.Example(features=tf.train.Features(feature={
                'image/encoded': _bytes_feature(encoded_jpg),
                'image/object/class/label': _int64_feature(classes[0]-1)
                }))
            writer.write(tf_example.SerializeToString())
    
        writer.close()
        print('Successfully created the TFRecords: {}'.format(output_path))

label_map = {v:int(k) for k,v in clt.label_map.items()}
create_record_files(label_map=label_map, record_dir=clt.record_dir, 
                    tfExample_generator=clt.tf_vars_generator)

Successfully created the TFRecords: mask-classification/Classification_COVID_Masks/0/records/train.record
Successfully created the TFRecords: mask-classification/Classification_COVID_Masks/0/records/eval.record


## Building our input pipeline

Now that our data are saved in an efficient format we want to load them as a ```tf.Data.Dataset``` object.

We have to define a feature_description dictionnary that follows the same structure as the one used to generate the ```tf.Example```.
 With this dictionnary we can define a parser for the ```tf.Example```

In [17]:
feature_description = {
      'image/encoded': tf.io.FixedLenFeature([], tf.string),
      'image/object/class/label': tf.io.FixedLenFeature([], tf.int64, default_value=0)}

def _parse_function(example_proto):
  # Parse the input `tf.Example` proto using the dictionary above.
    return tf.io.parse_single_example(example_proto, feature_description)

Let's create the ```tf.Data.dataset``` objects now by mapping the parser to the raw datasets !

In [18]:
raw_dataset = tf.data.TFRecordDataset(os.path.join(clt.record_dir,"train.record"))
train_dataset = raw_dataset.map(_parse_function)

raw_dataset = tf.data.TFRecordDataset(os.path.join(clt.record_dir,"eval.record"))
eval_dataset = raw_dataset.map(_parse_function)

Now that we have our dataset objects we want to do some pre-processing on them. 

For the label we will simply one_hot encode them. The images require a bit more attention. We will decode them, then resize them according to the size of the mobilenet_v2 model base input. Then we'll use the quite convenient ```mobilenet_v2.preprocess_input()``` function that cast the type to ```tf.float32``` and scale the pixels between -1 and 1.

In [19]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

def map_img_label(example_proto):
    img = tf.io.decode_jpeg(example_proto["image/encoded"], channels=3)
    img = tf.image.resize(img, (224,224))
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    label = example_proto["image/object/class/label"]
    label = tf.one_hot(label, depth=2)
    return (img,label)
    
train_set = train_dataset.map(map_img_label)
eval_set = eval_dataset.map(map_img_label)

Now we want to shuffle and batch our datasets. With a ```tf.Data.dataset``` it's fairly simple. We just need to apply the corresponding method with some arguments, namely the batch size and the buffer size for the shuffling.

We define some arbitrary values then apply the methods to our datasets. We do not use the ```repeat()``` method of a dataset because we want our epoch to end when the whole dataset is exhausted. 

If we added this method to both datasets, we would need to pass a ```steps_per_epoch``` and ``validation_steps`` to the ``fit`` method of our model when starting the training. Indeed, Tensorflow would not be able to know when to stop an epoch since the dataset will be infinitely repeating itself. 

At this stage we could add some data augmentation by mapping functions to the dataset. However we will not do it in this guide.

In [20]:
BATCH_SIZE = 16
SHUFFLE_BUFFER_SIZE = 50

train_set = train_set.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
eval_set = eval_set.batch(BATCH_SIZE)

# Model creation and training

## Model definition

Now that our input pipeline is built it's time to define our model. As said earlier we are going to do some transfer learning on the MobileNetV2 model. First let's import some keras functions and the MobileNetV2 model.

In [21]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

Our model will be made up of two sub-models. The first part will be the MobileNetV2 model with all of its layers frozen and we will plug on top of it a little headModel defined below.

In [22]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs = baseModel.input, outputs = headModel)
for layer in baseModel.layers:
    layer.trainable = False

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 0s 0us/step


We can print the summary of our model and see all the different layers as well as the number of trainable/non-trainable parameters.

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

## Compiling the model

We first define some arbitrary hyperparameters and a specific optimizer.

The next step is to compile our model. It's here that we can set the loss, metrics and optimizer chosen.

In [24]:
from tensorflow.keras.optimizers import Adam

INIT_LR = 1e-4
EPOCHS = 100

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

## Training the model

Let's start the training by using the ``fit`` method of our model. As arguments we simply specify a ``tf.Data`` train and validation sets and the number of epochs. 

In [27]:
History = model.fit(train_set,
    validation_data=eval_set,
    epochs=EPOCHS)

Epoch 1/100
6/6 [==============================] - 7s 1s/step - loss: 0.9751 - accuracy: 0.4444 - val_loss: 1.2585 - val_accuracy: 0.2174
Epoch 2/100
6/6 [==============================] - 0s 53ms/step - loss: 0.7508 - accuracy: 0.5889 - val_loss: 1.0552 - val_accuracy: 0.2609
Epoch 3/100
6/6 [==============================] - 0s 52ms/step - loss: 0.7165 - accuracy: 0.5778 - val_loss: 0.9004 - val_accuracy: 0.3913
Epoch 4/100
6/6 [==============================] - 0s 50ms/step - loss: 0.5446 - accuracy: 0.6889 - val_loss: 0.8053 - val_accuracy: 0.4348
Epoch 5/100
6/6 [==============================] - 0s 51ms/step - loss: 0.4937 - accuracy: 0.7778 - val_loss: 0.7375 - val_accuracy: 0.5652
Epoch 6/100
6/6 [==============================] - 0s 51ms/step - loss: 0.4568 - accuracy: 0.8111 - val_loss: 0.6541 - val_accuracy: 0.6522
Epoch 7/100
6/6 [==============================] - 0s 50ms/step - loss: 0.3990 - accuracy: 0.7778 - val_loss: 0.5629 - val_accuracy: 0.6522
Epoch 8/100
6/6 [=====

6/6 [==============================] - 0s 49ms/step - loss: 0.1430 - accuracy: 0.9444 - val_loss: 0.2514 - val_accuracy: 0.9130
Epoch 60/100
6/6 [==============================] - 0s 50ms/step - loss: 0.1144 - accuracy: 0.9667 - val_loss: 0.2362 - val_accuracy: 0.9130
Epoch 61/100
6/6 [==============================] - 0s 50ms/step - loss: 0.0363 - accuracy: 0.9889 - val_loss: 0.2136 - val_accuracy: 0.9130
Epoch 62/100
6/6 [==============================] - 0s 50ms/step - loss: 0.0496 - accuracy: 0.9778 - val_loss: 0.2135 - val_accuracy: 0.9130
Epoch 63/100
6/6 [==============================] - 0s 49ms/step - loss: 0.0287 - accuracy: 1.0000 - val_loss: 0.2168 - val_accuracy: 0.9130
Epoch 64/100
6/6 [==============================] - 0s 50ms/step - loss: 0.0297 - accuracy: 1.0000 - val_loss: 0.2215 - val_accuracy: 0.9130
Epoch 65/100
6/6 [==============================] - 0s 51ms/step - loss: 0.0364 - accuracy: 1.0000 - val_loss: 0.2238 - val_accuracy: 0.9130
Epoch 66/100
6/6 [========

# Saving

### Training logs

By default the fit method of a model returns a ```tf.keras.callbacks.History``` object which has some base logs from the training. We want to send those logs to the platform to see them on the dashboard. 


In [39]:
logs = {k:{"step": [str(e) for e in History.epoch], "value":[str(round(val, 3)) for val in v] } for k,v in History.history.items()}
clt.send_logs(logs)

Training logs have been sent to Picsell.ia Platform...
You can now inspect and showcase results on the platform.


This will create and send a dictionnary containing the logs in the right format for the platform to display them. 

### Checkpoint

We want to save a checkpoint of our model to, for example, continue the training later. To do this we want to create a ```Checkpoint``` object with our model and optimizer and save it.

In [40]:
checkpoint = tf.train.Checkpoint(optimizer=opt, model=model)
checkpoint.save(os.path.join(clt.checkpoint_dir, "model.ckpt"))
clt.send_checkpoints()

FileNotFoundError: No config file found

### Model

Saving a checkpoint is nice but mostly useful for future trainings. To directly save the model we simply need to apply the ```save``` method to our model and specify the directory.

In [42]:
model.save(clt.exported_model_dir)
clt.send_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mask-classification/Classification_COVID_Masks/0/exported_model/assets


NetworkError: Impossible to upload frozen graph to Picsell.ia backend